In [1]:
%load_ext Cython
%time
%cython

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


UsageError: Line magic function `%cython` not found (But cell magic `%%cython` exists, did you mean that instead?).


In [87]:
#import needed packages
from gensim.models import doc2vec

import re
import pandas as pd
import numpy as np
import copy

# nltk processing
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier


In [108]:
data = pd.read_csv("recTitleCatIngredCats.csv") 
dfTitle = data['title']
dfIngred = data['ingred']

In [117]:

def clean_data2(ds):
    arr = ['tablespoons','tablespoon','teaspoons','teaspoon','cups','cup','ounces','ounce','pounds','pound','pinches','pinch','tastes','taste','dashes','dash','/',' or ',' as ',' and ',
          '-','[',']','(',')','.','slices','sliced','slice', '0','1','2','3','4','5','6','7','8','9','juiced','diced','minced','chopped',' cans',' canned ',' can ', ' inches ', ' inch ',
          'package', 'finely', 'crushed', 'shredded', 'divided', ' melted ', ' jar ','thawed', ' cut ', ' to ', ' thick ', ' across ', ' bones ', 'optional', ' split ', ' torn ',
          ' into ', ' strips ', 'peeled',' pieces', ' lightly ', ' beaten ','ground black', 'scrubbed', 'jars', 'beaten','pitted','halved', 'thinly','as needed','large','medium','small'
          'roughly','fluid','drained','crumbled','rinsed','cubed','seeded','fresh',' plus ', ' more ', 'deboned', 'prepared', 'cubes', 'uncooked', 'cooked', 'grated', ' lean ','into chunks',
          'coarsely','seeded','freshly',' freshly', 'fresh', ' with ', 'liquid','quartered','lengthwise','small',' into ',' bite ', ' sized ', ' ly ', ' ed ', ' s ','cut into', ' up ',
          'trimmed','frozen','chunks','segments','softened',' cut ','pieces','broken',' rings',' torn ', ' ',"'"]
 
    for item in arr:
        ds = ds.str.replace(item,'')
     
    return ds


In [118]:
dfIngred = clean_data2(data['ingred'])

# remove measurements and anything else that is not a food ingredient
data['ingred'] = clean_data2(data['ingred'])

#data['title'] = clean_data(data['title'])

#dfIngred.to_csv('ingredClean.csv', sep=' ')

data['ingred']

0       TacoMeatFilling:,groundbeef,tacoseasoning,Batt...
1       groundbeef,Pace®TacoSeasoningMix,"Campbells®Co...
2       wholewheatbread,groundbeef,greenonion,,onion,s...
3       groundbeef,onionpowder,honeymustard,garlicpowd...
4       beefliver,milk,,butter,,Vidaliaonions,,allpurp...
5       lime,,garlic,driedoregano,groundcumin,chipotle...
6       quartwater,carrots,,potatoes,,onion,,salsa,hot...
7       groundbeef,porksausage,onion,,egg,,salt,pepper...
8       groundbeef,beefbouilloncube,water,clovesgarlic...
9       beefstewmeat,water,onion,,tomatoes,mixedvegeta...
10      beefeyeofroundroast,koshersalt,garlicpowder,pe...
11      groundbeef,Worcestershiresauce,tomatosauce,fri...
12      groundbeef,garlicpowder,sausageflavoredspaghet...
13      elbowmacaroni,groundbeef,onion,,condensedtomat...
14      groundbeef,onion,,clovesgarlic,,basil,driedore...
15      groundbeef,onion,celery,condensedvegetablesoup...
16      groundbeef,butterflavoredcrackers,onion,,eggs,...
17      ground

In [140]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Construct the required TF-IDF matrix by fitting and transforming the data
#tfidf_matrix = tfidf.fit_transform(data['ingred'])
tfidf_matrix = tfidf.fit_transform(data['title'])


#Output the shape of tfidf_matrix
tfidf_matrix.shape




(8110, 2738)

In [141]:
#ingred: 5,460 terms were used to describe 8110 records?


#With this matrix in hand, you can now compute a similarity score. There are several candidates for this; such as the euclidean, the Pearson and the cosine similarity scores. 

# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)



In [149]:

#Construct a reverse map of indices and movie titles
#indices = pd.Series(data.index, index=data['ingred']).drop_duplicates()
indices = pd.Series(data.index, index=data['title']).drop_duplicates()

indices.head()

#def findIngred(ingredient):
#    ingredient = 'groundbeef'
#    recs = []
#    for item in indices:
#        for subitem in item:
#            if subitem == ingredient:
#                recs.append(item)
#    return recs

#srecs = findIngred('groundbeef')
#srecs

#indices
#indices['beefeyeofroundroast,koshersalt,garlicpowder,pepper']
#idx = indices['beefeyeofroundroast,koshersalt,garlicpowder,pepper']
#sim_scores = list(enumerate(cosine_sim[idx]))
#sim_scores



title
1-Dish Taco Bake                   0
15-Minute Dinner Nachos Supreme    1
A Firefighter's Meatloaf           2
Aaron's Missouri Burger            3
Absolute Best Liver and Onions     4
dtype: int64

In [154]:
# Function that takes in ingredient as input and outputs most similar ingredients
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
#    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sorted(sim_scores, reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    recipe_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data['title'].iloc[recipe_indices]


In [155]:
get_recommendations("Aaron's Missouri Burger")


0    1-Dish Taco Bake
Name: title, dtype: object

In [12]:
stopwrds = stopwords.words('english')
# aux function to clean up text
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = re.sub('[^\w\s]',' ', sentence)
    sentence = re.sub('_',' ', sentence)
    sentence = re.sub('\d+',' ', sentence)
    cleaned = ' '.join([w for w in sentence.split() if not w in stopwrds])
    cleaned = ' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    cleaned = ' '.join([w for w in cleaned.split() if not len(w)<=2 ])
    cleaned = cleaned.strip()
    return cleaned

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  Searched in:
    - '/Users/cassc042/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/anaconda3/nltk_data'
    - '/anaconda3/lib/nltk_data'
**********************************************************************


In [76]:
# clean up review docs, add utf8 encoding
#review_docs['textClean'] = review_docs.apply(lambda row: cleaning_text(row['text'].encode("utf8")), axis=1)


In [77]:
# define a few functions for doc2vec processing
def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

# MyDocs reading from a data frame
class MyDocs(object):
    def __iter__(self):
        dfList = dfIngred.values.tolist()
        for i in range(len(dfList)):
            yield doc2vec.TaggedDocument(words=split_sentence(dfIngred[i]), tags=['%s' % dfIngred[i]])
            
            

In [78]:
# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, vector_size = 200, window = 8, min_count = 5, workers = 4)
model.save("title.model")

In [79]:
# testing similar words
print(model.most_similar(positive=["ground","beef"], negative=["slow"], topn=5))


[('lean', 0.6304351687431335), ('minced', 0.6254710555076599), ('crushed', 0.6162852048873901), ('onion', 0.6086111068725586), ('paprika', 0.6044589877128601)]


/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [80]:
tokens = "ground beef".split()

new_vector = model.infer_vector(tokens)

sims = model.docvecs.most_similar([new_vector]) #gives you top 10 document tags and their cosine similarity

In [81]:
sims

[('  margarine, softened,   peanut butter,   white sugar,   packed brown sugar,  egg,   vanilla extract, /  baking soda,   salt,  /all-purpose flour',
  0.9509450197219849),
 ('  unsweetened almond milk,   frozen blackberries,   frozen blueberries,   coconut butter,   honey,   chia seeds',
  0.9504584074020386),
 ('  frozen pineapple chunks,   frozen mango chunks,   toasted coconut vanilla yogurt,   milk',
  0.950424313545227),
 (' /all-purpose flour,   baking soda,   baking powder,   butter, softened,  white sugar,  egg,   vanilla extract',
  0.9490064382553101),
 ('  fresh lime juice,   white wine vinegar,   salt,   freshly ground black pepper,   ground ginger,   dried basil,   dried thyme,   dried parsley,   hot pepper sauce,   cayenne pepper,   vegetable oil,  fresh swordfish fillets',
  0.9485344886779785),
 ('  soy sauce,   honey,   distilled white vinegar,   ground ginger,   garlic powder,   vegetable oil,  flank steak',
  0.9473416805267334),
 ('all-purpose flour,   salt, bakin

In [82]:
#save a pickle file
import pickle

with open('models/model_ingred_only.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [83]:
data.head()

,title,ingred,rating
0,Wins Shrimp and Spaghetti,"uncooked spaghetti, butter, Creole-style s...",4.41
1,Cheez-It® Fried Tofu Pita Pocket,"package extra-firm tofu, cut into -inch cub...",4.00
2,King Crab Appetizers,"packages refrigerated biscuit dough, pac...",4.37
3,Spinach Basil Pasta Salad,"package bow tie pasta, package spinach l...",4.67
4,Butternut Squash Kugel I,"package frozen butternut squash, cubed, m...",4.17


In [84]:
# get a unique summary corpus
#summary_docs = games[['uniqueID','summary']].append(movies[['uniqueID', 'summary']]).append(tv[['uniqueID','summary']])
#summary_docs.head(5)

In [85]:
# define class for summary Doc2vec

# MyDocs reading from a data frame
class MyDocs_summary(object):
    def __iter__(self):
        for i in range(data.shape[0]):
            yield doc2vec.TaggedDocument(words=split_sentence(data.iloc[i,0]), tags=['%s' % data.iloc[i,1]])
            

In [86]:
# Train the doc2vec model on summary
mydocs_summary = MyDocs_summary()
model_summary = doc2vec.Doc2Vec(mydocs_summary, vector_size = 200, window = 8, min_count = 5, workers = 4)
model_summary.save("summary.model")

In [90]:
print(model_summary.wv.most_similar(positive=["stew", "beef"], negative=["slow"], topn=5))



[('smoothie', 0.9997438192367554), ('zucchini', 0.9997418522834778), ('sauce', 0.9997351169586182), ('shrimp', 0.9997339844703674), ('lemon', 0.9997339248657227)]


In [91]:
# Combining both summary and critic reviews doc2vec :model_summary and model 
summary_ingred_docvecs = np.zeros(shape=(len(model_summary.docvecs),400))
for i in range(len(model_summary.docvecs)):
    summary_ingred_docvecs[i] = np.concatenate((model_summary.docvecs[i],model.docvecs[i]), axis = 0)
    
    

In [94]:
#save pickle file
import pickle
with open('summary_ingred_docvecs.pickle', 'wb') as handle:
    pickle.dump(summary_ingred_docvecs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [95]:
import numpy as np
import pandas as pd
import pickle
import copy
####### TEST REC FROM HERE
# For the app, the code for the recommendation would load a pickle from this cell

#to load a pickle file
with open('summary_ingred_docvecs.pickle', 'rb') as f:
    summary_ingred_docvecs = pickle.load(f)
    
# Auxiliary functions for simple recommendation system 

# Calculate cosine similarity between two vecotrs 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 

# return top_n values from a list
def top_n_index(l,n):
    return sorted(range(len(l)), key=lambda i: l[i])[-(n+1):-1] #-1 to take off the own product from the returned index list

# return a list of unique_id for a given category["games", "movies","tv"]
def category_id_range(category_list):
    # range of ids for each category
    games_range = range(1,20417)
    movies_range = range(20417, 25887)
    tv_range = range(25887, 27865)
    
    category_range = []
    for i in category_list:
        if i=="games":
            category_range = category_range + games_range
        elif i=="movies":
            category_range = category_range + movies_range
        else:
            category_range = category_range + tv_range
    
    return category_range

def content_recommend(item_id, top_n, inputs):
    input_vec = inputs[item_id - 1]
    
    #compute similarity array
    sim_array = map(lambda v: cossim(input_vec, v), inputs)
    
    # recommendation's index (set to 50 to get enough to filter out later)
    recommendation_index = top_n_index(sim_array, 500)
    
    # recommendation's unique id
    recommendation_unique_id = [i+1 for i in recommendation_index]
    
    # recommendation's cossim values
    recommendation_cossim = [sim_array[i] for i in recommendation_index]
    
    top_products = zip(recommendation_unique_id, recommendation_cossim)
    
    # get the range of unique id for a given category prefered by user
    #category_range = category_id_range(category_list)
    
    result = [i for i in top_products]  # if i[0] in category_range]
    
    return result[-top_n:]